In [1]:
import pandas as pd
import numpy as np
import subprocess
import re
from subprocess import Popen, PIPE

In [2]:
sample_sub = pd.read_csv('sample_submission.csv')
sample_sub.tail()

,id,EAP,HPL,MWS
8387,id11749,0.403494,0.287808,0.308698
8388,id10526,0.403494,0.287808,0.308698
8389,id13477,0.403494,0.287808,0.308698
8390,id13761,0.403494,0.287808,0.308698
8391,id04282,0.403494,0.287808,0.308698


In [3]:
def write_content_txt_file(content):
    clean_txt_file()
    with open('test.txt', 'w') as f:
        f.write(content)
    return None

def clean_txt_file():
    f = open('test.txt', 'r+')
    f.truncate()
    return None

## Data cleaning functions
def remove_label(text):
    txt = re.sub('__label__', '', text)
    return txt

def remove_new_line(text):
    txt = re.sub('\n', '', text)
    return txt

In [4]:
test = pd.read_csv('test.csv')
test.tail()

,id,text
8387,id11749,All this is now the fitter for my purpose.
8388,id10526,I fixed myself on a wide solitude.
8389,id13477,It is easily understood that what might improv...
8390,id13761,"Be this as it may, I now began to feel the ins..."
8391,id04282,"Long winded, statistical, and drearily genealo..."


## Preprocessing test input

In [5]:
import re
import nltk
from nltk.corpus import stopwords
import gensim
nltk.download('stopwords')
stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def transformText(text):
    
    #stops = set(stopwords.words("english"))
    
    # Convert text to lower
    text = text.lower()
    
    # Removing non ASCII chars    
    text = re.sub(r'[^\x00-\x7f]',r' ',text)
    
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    
    # Removing all the stopwords
    #filtered_words = [word for word in text.split() if word not in stops]
   
    filtered_words = [word for word in text.split()]

    # Removing all the tokens with lesser than 3 characters
    filtered_words = gensim.corpora.textcorpus.remove_short(filtered_words, minsize=3)
    
    # Preprocessed text after stop words removal
    text = " ".join(filtered_words)
    
    # Remove the punctuation
    #text = gensim.parsing.preprocessing.strip_punctuation2(text)
    
    # Strip all the numerics
    text = gensim.parsing.preprocessing.strip_numeric(text)
    
    #text = gensim.parsing.strip_punctuation(text)
    
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)

    # Stemming
    #text = gensim.parsing.preprocessing.stem_text(text)
    return text

In [7]:
train = pd.read_csv('train.csv')
train.tail()

,id,text,author
19574,id17718,"I could have fancied, while I looked at it, th...",EAP
19575,id08973,The lids clenched themselves together as if in...,EAP
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP
19577,id17513,"For an item of news like this, it strikes us i...",EAP
19578,id00393,"He laid a gnarled claw on my shoulder, and it ...",HPL


In [8]:
train['text_processed']=train['text'].apply(lambda x: transformText(x))
train.tail()

,id,text,author,text_processed
19574,id17718,"I could have fancied, while I looked at it, th...",EAP,"could have fancied, while looked it, that some..."
19575,id08973,The lids clenched themselves together as if in...,EAP,the lids clenched themselves together spasm.
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP,"mais faut agir that say, frenchman never faint..."
19577,id17513,"For an item of news like this, it strikes us i...",EAP,"for item news like this, strikes was very cool..."
19578,id00393,"He laid a gnarled claw on my shoulder, and it ...",HPL,"laid gnarled claw shoulder, and seemed that it..."


## Generating fastText train file

In [78]:
def create_fasttext_label(file):
    txt = "__label__"+file
    return txt

In [79]:
with open("spooky_4" + '.txt','w') as txtfile:
    for i in range(len(train.id)):
        txtfile.write(create_fasttext_label(train['author'][i]) + " " + train['text_processed'][i] + "\n")
!head -n 5 spooky_4.txt

__label__EAP this process, however, afforded means ascertaining the dimensions dungeon; might make its circuit, and return the point whence set out, without being aware the fact; perfectly uniform seemed the wall.
__label__HPL never once occurred that the fumbling might mere mistake.
__label__EAP his left hand was gold snuff box, from which, capered down the hill, cutting all manner fantastic steps, took snuff incessantly with air the greatest possible self satisfaction.
__label__MWS how lovely spring looked from windsor terrace the sixteen fertile counties spread beneath, speckled happy cottages and wealthier towns, all looked former years, heart cheering and fair.
__label__HPL finding nothing else, not even gold, the superintendent abandoned his attempts; but perplexed look occasionally steals over his countenance sits thinking his desk.


In [80]:
!wc spooky_4.txt

  19579  432425 2882216 spooky_4.txt


In [81]:
## Total 19579 lines
print("train set {}".format(int(19579*0.8)))
print("test set {}".format(19579-15660))

train set 15663
test set 3919


In [82]:
## Train set
!head -n 15660 spooky_4.txt > spooky_4.train
## Test set
!tail -n 3919 spooky_4.txt > spooky_4.valid

## Training fastText model

In [ ]:
#!fasttext supervised -input spooky_4.train -output model_3_subword -epoch 50000 -lr 0.2 \
#                     \-minCount 3 -wordNgrams 2 -loss softmax -dim 20 -ws 3
# loss = , acc = 0.826, 
#!fasttext supervised -input spooky_4.train -output model_4 -epoch 50000 -lr 0.5 -loss softmax -dim 15
# loss = 0.001737, acc = 0.789
#!fasttext supervised -input spooky_4.train -output model_4 -epoch 50000 -lr 1 -loss softmax -dim 15
# loss: 0.001131, acc = 0.79
#!fasttext supervised -input spooky_4.train -output model_4 \
#                        -epoch 50000 -lr 0.8 -loss softmax \
#                        -wordNgrams 3 -dim 20 -ws 3 -minCount 2
# loss: 0.001478, acc = 0.79
!fasttext supervised -input spooky_4.train -output model_4 \
                        -epoch 100000 -lr 0.8 -loss softmax \
                        -wordNgrams 2 -dim 20 -ws 3 -minCount 2

Read 0M words
Number of words:  19463
Number of labels: 3


Progre

Progress: 0.6%  words/sec/

Progress: 0.6%  words/sec/thread: 2044249  l

Progress: 0.6%  words/sec/thread: 2044385  lr: 0.795316  loss: 0.006460  e

...

Progress: 5.5%  words/sec/thread: 20504

Progress: 6.1%  words/sec/thread: 2050224  lr: 0.751156  loss: 0.002243  eta: 0h22m ss: 0.002334  eta: 0h23m h23m r: 0.756128  loss: 0.002333  eta: 0h23m   words/sec/thread: 2050359  lr: 0.756111  loss: 0.002332  eta: 0h23m c/thread: 2050339  lr: 0.756101  loss: 0.002332  eta: 0h23m 2  loss: 0.002330  eta: 0h23m ds/sec/thread: 2050244  lr: 0.756002  loss: 0.002330  eta: 0h23m 755920  loss: 0.002329  eta: 0h23m ss: 0.002328  eta: 0h23m 2050149  lr: 0.755904  loss: 0.002328  eta: 0h23m r: 0.755844  loss: 0.002327  eta: 0h23m   words/sec/thread: 2050040  lr: 0.755831  loss: 0.002326  eta: 0h23m h23m ress: 5.5%  words/sec/thread: 2049959  lr: 0.755760  loss: 0.002325  eta: 0h23m   words/sec/thread: 2049951  lr: 0.755756  loss: 0.002325  eta: 0h23m 2049929  lr: 0.755741  loss: 0.002324  eta: 0h23m r: 0.755680  loss: 0.002323  eta: 0h23m   words/sec/thread: 2049765  lr: 0.755669  loss: 0.002323  eta: 0h23m c/thread: 2049750  lr: 0.755661  loss: 0.002322  eta: 0h23m 7  loss: 0.002321  eta: 0h

Progress: 6.8%  words/sec/thread: 2052469  lr: 0.745557  loss: 0.002162  eta: 0h22m  6.1%  words/sec/thread: 2050184  lr: 0.751080  loss: 0.002242  eta: 0h22m ss: 0.002242  eta: 0h22m 2050176  lr: 0.751070  loss: 0.002241  eta: 0h22m 750970  loss: 0.002240  eta: 0h22m ss: 0.002240  eta: 0h22m 02  lr: 0.750875  loss: 0.002239  eta: 0h22m  6.1%  words/sec/thread: 2050279  lr: 0.750866  loss: 0.002239  eta: 0h22m ss: 0.002238  eta: 0h22m 2050252  lr: 0.750848  loss: 0.002238  eta: 0h22m  6.2%  words/sec/thread: 2050290  lr: 0.750770  loss: 0.002237  eta: 0h22m 2050312  lr: 0.750755  loss: 0.002236  eta: 0h22m  6.2%  words/sec/thread: 2050325  lr: 0.750687  loss: 0.002235  eta: 0h22m 2050330  lr: 0.750678  loss: 0.002235  eta: 0h22m r: 0.750603  loss: 0.002234  eta: 0h22m   words/sec/thread: 2050370  lr: 0.750595  loss: 0.002234  eta: 0h22m .002233  eta: 0h22m r: 0.750503  loss: 0.002233  eta: 0h22m  6.2%  words/sec/thread: 2050437  lr: 0.750492  loss: 0.002232  eta: 0h22m .002232  eta: 0h

Progress: 7.5%  words/sec/thread: 2052364  lr: 0.739891  loss: 0.002087  eta: 0h22m ss: 0.002161  eta: 0h22m 2052479  lr: 0.745530  loss: 0.002161  eta: 0h22m r: 0.745460  loss: 0.002160  eta: 0h22m 2  loss: 0.002160  eta: 0h22m h22m r: 0.745378  loss: 0.002159  eta: 0h22m   words/sec/thread: 2052550  lr: 0.745368  loss: 0.002158  eta: 0h22m .002158  eta: 0h22m 33  lr: 0.745282  loss: 0.002158  eta: 0h22m 745272  loss: 0.002157  eta: 0h22m   words/sec/thread: 2052535  lr: 0.745270  loss: 0.002157  eta: 0h22m 2052529  lr: 0.745267  loss: 0.002157  eta: 0h22m 745171  loss: 0.002156  eta: 0h22m   words/sec/thread: 2052604  lr: 0.745163  loss: 0.002155  eta: 0h22m h22m r: 0.745074  loss: 0.002155  eta: 0h22m  6.9%  words/sec/thread: 2052612  lr: 0.745063  loss: 0.002154  eta: 0h22m ds/sec/thread: 2052605  lr: 0.745055  loss: 0.002154  eta: 0h22m 744954  loss: 0.002153  eta: 0h22m   words/sec/thread: 2052560  lr: 0.744946  loss: 0.002153  eta: 0h22m .002153  eta: 0h22m 2  loss: 0.002152  et

Progress: 8.0%  words/sec/thread: 2054447  lr: 0.735964  loss: 0.002041  eta: 0h22m 739792  loss: 0.002086  eta: 0h22m   words/sec/thread: 2052332  lr: 0.739786  loss: 0.002086  eta: 0h22m 71  lr: 0.739675  loss: 0.002086  eta: 0h22m ress: 7.5%  words/sec/thread: 2052372  lr: 0.739667  loss: 0.002085  eta: 0h22m 4  loss: 0.002085  eta: 0h22m ds/sec/thread: 2052358  lr: 0.739654  loss: 0.002085  eta: 0h22m r: 0.739545  loss: 0.002084  eta: 0h22m  7.6%  words/sec/thread: 2052482  lr: 0.739542  loss: 0.002084  eta: 0h22m ds/sec/thread: 2052476  lr: 0.739534  loss: 0.002084  eta: 0h22m 2052483  lr: 0.739525  loss: 0.002083  eta: 0h22m r: 0.739440  loss: 0.002083  eta: 0h22m  7.6%  words/sec/thread: 2052497  lr: 0.739434  loss: 0.002083  eta: 0h22m 2052514  lr: 0.739425  loss: 0.002082  eta: 0h22m 739334  loss: 0.002082  eta: 0h22m   words/sec/thread: 2052580  lr: 0.739331  loss: 0.002082  eta: 0h22m 97  lr: 0.739241  loss: 0.002081  eta: 0h22m 739234  loss: 0.002081  eta: 0h22m   words/sec

## Testing fastText model

In [9]:
!fasttext test model_4.bin spooky_4.valid

N	3919
P@1	0.79
R@1	0.79
Number of examples: 3919


## Generating output

In [61]:
test['text_processed']=test['text'].apply(lambda x: transformText(x))
test.head()

,id,text,text_processed
0,id02310,"Still, as I urged our leaving Ireland with suc...",still urged our leaving ireland with such inqu...
1,id24541,"If a fire wanted fanning, it could readily be ...",fire wanted fanning could readily fanned with ...
2,id00134,And when they had broken down the frail door t...,and when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...,while was thinking how should possibly manage ...
4,id04081,I am not sure to what limit his knowledge may ...,not sure what limit his knowledge may extend


In [62]:
with(open('test_frases.txt', 'w')) as f:
    for i in range(len(test)):
        f.write(test['text_processed'][i]+"\n")
!head -n 5 test_frases.txt

still urged our leaving ireland with such inquietude and impatience father thought best yield 
fire wanted fanning could readily fanned with newspaper and the government grew weaker have doubt that leather and iron acquired durability proportion for very short time there was not pair bellows all rotterdam that ever stood need stitch required the assistance hammer 
and when they had broken down the frail door they found only this two cleanly picked human skeletons the earthen floor and number singular beetles crawling the shadowy corners 
while was thinking how should possibly manage without them one actually tumbled out head and rolling down the steep side the steeple lodged the rain gutter which ran along the eaves the main building 
not sure what limit his knowledge may extend 


In [63]:
!ls -lah 

total 6.2G
drwxrwxr-x 3 ubuntu ubuntu 4.0K Nov  1 13:07 .
drwxrwxr-x 5 ubuntu ubuntu 4.0K Oct 27 12:06 ..
-rw-rw-r-- 1 ubuntu ubuntu 2.8K Oct 30 21:53 baseline_clasifier_2.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  89K Nov  1 11:44 baseline_clasifier.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 3.1M Nov  1 12:51 dataset_explore.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 253K Oct 30 16:33 generate_fasttext_predictions_submission_0.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 2.2M Oct 30 18:55 generate_fasttext_predictions_submission_2.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 1.4M Nov  1 13:07 generate_fasttext_predictions_submission.ipynb
drwxr-xr-x 2 ubuntu ubuntu 4.0K Oct 30 21:52 .ipynb_checkpoints
-rw-rw-r-- 1 ubuntu ubuntu 3.4G Oct 30 14:54 model_1_subword.bin
-rw-rw-r-- 1 ubuntu ubuntu 616M Oct 30 14:55 model_1_subword.vec
-rw-rw-r-- 1 ubuntu ubuntu 154M Nov  1 13:07 model_3_subword.bin
-rw-rw-r-- 1 ubuntu ubuntu 2.0M Nov  1 13:07 model_3_subword.vec
-rw------- 1 ubuntu ubuntu 2.0G Oct 27 19:04 nohup.out
-rw-rw-r-- 1 ubuntu ubuntu

In [12]:
my_sub = pd.DataFrame(columns={'id', 'EAP','HPL', 'MWS'})
my_sub=my_sub[['id', 'EAP','HPL', 'MWS']]
my_sub

,id,EAP,HPL,MWS


In [64]:
fast = Popen(['fasttext', 'predict-prob', 'model_3_subword.bin','test_frases.txt', '3'], stdin=PIPE, stdout=PIPE, stderr=PIPE)
output, err = fast.communicate()
#print(output)
reply = "".join(output.decode('utf8')).split("\n")
for i in range(len(reply)):
    if re.search('__label__',reply[i]):
        reply[i]=remove_label(reply[i])
    if re.search('\n',reply[i]):
        reply[i]=remove_new_line(reply[i])

## Clearning empty replies
reply[:] = [item for item in reply if item !='']
outer =[]
for i in range(len(reply)):
    outer.append(reply[i].split(" "))
    
for i in range(len(outer)):
    for j in range(len(outer[i])):
        if re.search('EAP', outer[i][j]):
            EAP = outer[i][j+1]
        if re.search('MWS', outer[i][j]):
            MWS = outer[i][j+1]
        if re.search('HPL', outer[i][j]):
            HPL = outer[i][j+1]
    my_sub.loc[i] = [test['id'][i], EAP, HPL, MWS]

In [68]:
my_sub.tail()

,id,EAP,HPL,MWS
8387,id11749,0.998047,1.95313e-08,1.95313e-08
8388,id10526,1.95313e-08,1.95313e-08,1
8389,id13477,0.980469,1.95313e-08,0.015625
8390,id13761,1.95313e-08,1.95313e-08,0.998047
8391,id04282,0.328125,0.669922,1.95313e-08


In [69]:
my_sub.to_csv('roberto_10.csv',index=False)
!ls -lah

total 6.2G
drwxrwxr-x 3 ubuntu ubuntu 4.0K Nov  1 13:10 .
drwxrwxr-x 5 ubuntu ubuntu 4.0K Oct 27 12:06 ..
-rw-rw-r-- 1 ubuntu ubuntu 2.8K Oct 30 21:53 baseline_clasifier_2.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  89K Nov  1 11:44 baseline_clasifier.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 3.1M Nov  1 12:51 dataset_explore.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 253K Oct 30 16:33 generate_fasttext_predictions_submission_0.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 2.2M Oct 30 18:55 generate_fasttext_predictions_submission_2.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 1.4M Nov  1 13:09 generate_fasttext_predictions_submission.ipynb
drwxr-xr-x 2 ubuntu ubuntu 4.0K Oct 30 21:52 .ipynb_checkpoints
-rw-rw-r-- 1 ubuntu ubuntu 3.4G Oct 30 14:54 model_1_subword.bin
-rw-rw-r-- 1 ubuntu ubuntu 616M Oct 30 14:55 model_1_subword.vec
-rw-rw-r-- 1 ubuntu ubuntu 154M Nov  1 13:07 model_3_subword.bin
-rw-rw-r-- 1 ubuntu ubuntu 2.0M Nov  1 13:07 model_3_subword.vec
-rw------- 1 ubuntu ubuntu 2.0G Oct 27 19:04 nohup.out
-rw-rw-r-- 1 ubuntu ubuntu